In [1]:
#Links Uteis:
#https://docs.microsoft.com/pt-br/sql/connect/python/pyodbc/python-sql-driver-pyodbc?view=sql-server-ver15
#https://docs.microsoft.com/pt-br/sql/machine-learning/data-exploration/python-dataframe-sql-server?view=sql-server-ver15

In [2]:
#Importando Bibliotecas
import pyodbc
import pandas as pd

In [3]:
#Fazendo a conexao com o banco que vou extrair os dados 
dados_conexao =(
    "Driver={SQL Server};"
    "Server=DESKTOP-0O9J9LL;"
    "Database=MestreElectronics;"
    )

conexao = pyodbc.connect(dados_conexao)

# Criando Dimensão de Produto

In [4]:
#Lendo as tabelas do banco com o pandas
produto_df = pd.read_sql('SELECT * FROM MestreElectronics.dbo.product', conexao)
categoria_df = pd.read_sql('SELECT * FROM MestreElectronics.dbo.productcategory', conexao)
subcategoria_df = pd.read_sql('SELECT * FROM MestreElectronics.dbo.productsubcategory', conexao)

In [5]:
#Mesclando as tabelas com as informações uteis
produto_df = produto_df[['ProductKey', 'ProductName', 'ProductSubcategoryKey', 'BrandName', 'ClassName']].merge(subcategoria_df)
produto_df = produto_df.merge(categoria_df)

In [6]:
#Deletando colunas chaves que não serão mais utilizadas 
produto_df = produto_df.drop(['ProductSubcategoryKey', 'ProductCategoryKey'], axis = 1)

In [7]:
#Criando cursor (Necessario para executar consultas em SQL)
cursor = conexao.cursor()

In [8]:
#Tratativa de erro para quando rodar o codigo
deletar_tabela = 'USE DW_MestreElectronics drop table DM_Produto'
cursor.execute(deletar_tabela)
cursor.commit()

In [9]:
#Criação do banco de dados, e comentei o codigo para não ficar criando toda vez que roda
#criar_database = 'CREATE DATABASE DW_MestreElectronics'
#cursor.execute(criar_database)
#cursor.commit()

In [10]:
#Criando tabela Dimensão Produto
criar_tabela = '''USE DW_MestreElectronics
CREATE TABLE DM_Produto(
    ProductKey int,
    ProductName varchar(100),
    BrandName varchar(50),
    ClassName varchar(50),
    ProductSubcategoryName varchar(50),
    ProductCategoryName varchar(50) )'''

cursor.execute(criar_tabela)
cursor.commit()

In [11]:
# Para inserir linha por linha na tabela que eu criei no banco de dados
for index, row in produto_df.iterrows():
     cursor.execute(''' USE DW_MestreElectronics
     INSERT INTO DM_Produto (ProductKey,ProductName,BrandName,ClassName,ProductSubcategoryName,ProductCategoryName) 
     values(?,?,?,?,?,?)''', row.ProductKey, row.ProductName, row.BrandName, row.ClassName, row.ProductSubcategoryName, row.ProductCategoryName)

cursor.commit()

# Criando Dimensão de Cliente

In [12]:
clientes_df  = pd.read_sql('SELECT * FROM MestreElectronics.dbo.Customer', conexao)
paises_df = pd.read_sql('SELECT * FROM MestreElectronics.dbo.Geography', conexao)

In [14]:
# Concatenando os nomes dos clientes em uma unica coluna 
clientes_df['NomeCompleto'] = clientes_df['FirstName'].str.cat(clientes_df['LastName'],sep=" ")
#deletando colunas
clientes_df = clientes_df.drop(['FirstName', 'LastName','MaritalStatus','EmailAddress','Education'], axis = 1)
#Deletando Colunas
paises_df = paises_df.drop(['GeographyType','CityName'], axis = 1)

In [32]:
clientes_df = clientes_df.merge(paises_df)

In [34]:
clientes_df = clientes_df.drop('GeographyKey', axis = 1)

In [35]:
clientes_df

,CustomerKey,BirthDate,Gender,Occupation,CustomerType,CompanyName,NomeCompleto,ContinentName,StateProvinceName,RegionCountryName
0,9896,1966-12-11,M,Clerical,Person,None,Ruben Jimenez,Europe,Seine (Paris),France
1,9911,1966-02-21,F,Professional,Person,None,Carolyn Malhotra,Europe,Seine (Paris),France
2,9912,1966-08-02,F,Professional,Person,None,Katrina Raje,Europe,Seine (Paris),France
3,9936,1977-10-14,F,Clerical,Person,None,Desiree Jimenez,Europe,Seine (Paris),France
4,9961,1962-03-05,M,Clerical,Person,None,Darrell Beck,Europe,Seine (Paris),France
...,...,...,...,...,...,...,...,...,...,...
18864,19141,None,None,None,Company,ThimphuCompany,NaN,Asia,Thimphu District,Bhutan
18865,19142,None,None,None,Company,TokyoCompany,NaN,Asia,Kanto,Japan
18866,19143,None,None,None,Company,UrumqiCompany,NaN,Asia,Xinjiang,China
18867,19144,None,None,None,Company,YerevanCompany,NaN,Asia,Armenia,Armenia
